В целом, в предобработанной версии не хватает только лемматизации.

In [ ]:
# Imports 

import os
import pandas as pd
import nltk
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
from nltk.corpus import stopwords
from tqdm import tqdm
import hw.tg_utils

In [7]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
sent_detector.PUNCTUATION


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/paperspace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/paperspace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/paperspace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


(';', ':', ',', '.', '!', '?')

## Часть 3. [5 баллов] Классификация текстов
Задача классификации формулируется так: данный фрагмент фанфика описывают какую-то ситуацию, которая предшествует произнесению заклинания. Требуется по тексту предсказать, какое именно заклинание будет произнесено. Таким образом, заклинание - это фактически метка класса. Основная мера качества – macro $F_1$.
Обучите несколько классификаторов и сравните их между собой. Оцените качество классификаторов на частых и редких классах. Какие классы чаще всего оказываются перепутаны? Связаны ли ошибки со смыслом заклинаний?

Используйте фрагменты из множества train для обучения, из множества dev для отладки, из множества test – для тестирования и получения итоговых результатов. 

1. [1 балл] Используйте fastText в качестве baseline-классификатора.
2. [2 балла] Используйте сверточные  или реккурентные сети в качестве более продвинутого классификатора. Поэкспериментируйте с количеством и размерностью фильтров, используйте разные размеры окон, попробуйте использовать $k$-max pooling. 
3. [2 балла] Попробуйте расширить обучающее множество за счет аугментации данных. Если вам понадобится словарь синонимов, можно использовать WordNet (ниже вы найдете примеры).

[бонус] Используйте результат max pooling'а как эмбеддинг входного текста. Визуализируйте эмбеддинги 500-1000 предложений из обучающего множества и изучите свойства получившегося пространства.

[бонус] Используйте ваш любимый классификатор и любые (честные) способы повышения качества классификации и получите macro $F_1$ больше 0.5.

In [22]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from sklearn.metrics import f1_score
from torchtext.vocab import Vocab
from torchtext.vocab import vocab as torchVocab
import gensim
import numpy as np
from tqdm import tqdm

In [7]:
import pandas as pd

train_df = pd.read_csv("./hpac_corpus/hpac_training_128.tsv", sep="\t", header=None, names=["score", "spell", "sent"])
dev_df = pd.read_csv("./hpac_corpus/hpac_dev_128.tsv", sep="\t", header=None, names=["score", "spell", "sent"])
test_df = pd.read_csv("./hpac_corpus/hpac_test_128.tsv", sep="\t", header=None, names=["score", "spell", "sent"])

In [8]:
from torchtext.transforms import RegexTokenizer
from nltk.tokenize import RegexpTokenizer
from string import punctuation

def preprocess_data(df):
    punct = set(list(punctuation) + ["``", "--", "n", "''", "..."])

    tokenizer = RegexpTokenizer(r"\w+")
    def preprocess_sent(s):
        tokens = tokenizer.tokenize(s)
        res = []
        for tok in tokens:
            if tok not in punct:
                res.append(tok)
        return " ".join(res)
    df["preprocess_sent"] = df["sent"].apply(preprocess_sent)
    return df

In [9]:
train_df = preprocess_data(train_df)
test_df = preprocess_data(test_df)
dev_df = preprocess_data(dev_df)

In [10]:
w2v = gensim.models.fasttext.load_facebook_vectors("/home/paperspace/hw-1-nlp-hse-2022-takkat14/hw/word_reprs_fasttext.bin")

In [11]:
labels = set(train_df["spell"].values).union(set(test_df["spell"].values)).union(set(dev_df["spell"].values))
label2idx = {label:i for i, label in enumerate(labels)}
idx2label = {label2idx[label]:label for label in labels}

In [12]:
from torch.utils.data import IterableDataset


class HPAC_Dataset(IterableDataset):
    def __init__(self, df, label_col="spell", sent_col="preprocess_sent", vector_model=w2v, aug=False, aug_prob=0.15, emb_dim=100, max_len=256, label2idx=label2idx):
        self.data = df[[label_col, sent_col]]
        self.model = vector_model
        self.aug = aug
        self.max_len = max_len
        self.emb_dim = emb_dim 
        self.label2idx = label2idx     
        self.aug_prob = aug_prob  
        
        
    def __iter__(self):
        for i, sample in self.data.iterrows():
            features = np.zeros((self.max_len, self.emb_dim))
            for j, word in enumerate(sample[1].split(" ")):
                if j == self.max_len:
                    break
                if not self.aug:
                    features[j] = self.model.get_vector(word)
                else:
                    if i % 100 == 0 and np.random.rand() > 1 - self.aug_prob:
                        choose = np.random.choice(10)
                        if np.random.rand() > 0.5:
                            features[j] = self.model.get_vector(self.model.most_similar(positive=[word])[choose][0])
                        else:
                            features[j] = self.model.get_vector(self.model.most_similar(negative=[word])[choose][0])
            
            yield torch.tensor(features.astype(np.float32)), label2idx[sample[0]]


In [13]:
train_dataset = HPAC_Dataset(train_df)
val_dataset = HPAC_Dataset(dev_df)

In [14]:
class CNNModel(nn.Module):
    def __init__(self, kernel_size=3, padding=1, stride=1):
        super().__init__()

        self.model = nn.Sequential(
            nn.Conv1d(256, 100, kernel_size=kernel_size, padding=padding),
            nn.ReLU(),
            nn.Conv1d(100, 16, kernel_size=kernel_size, padding=kernel_size),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(1664, 256),
            nn.ReLU(),
            nn.Linear(256, 85)
            )


    def forward(self, x):
        return self.model(x)


In [15]:
import torch

model = CNNModel()
optimizer = optim.Adam(model.parameters())
device = 'cpu'  if torch.cuda.is_available() else torch.cuda.current_device()

model.to(device)
    

CNNModel(
  (model): Sequential(
    (0): Conv1d(256, 100, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(100, 16, kernel_size=(3,), stride=(1,), padding=(3,))
    (3): ReLU()
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=1664, out_features=256, bias=True)
    (6): ReLU()
    (7): Linear(in_features=256, out_features=85, bias=True)
  )
)

In [16]:
BATCH_SIZE = 64

train_loader = DataLoader(train_dataset, pin_memory=True, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_dataset, shuffle=False, pin_memory=True, batch_size=BATCH_SIZE)
    

In [25]:
def train_eval_loop(model, opt, tdl, vdl, num_epochs=5, device=device):
    mean_train_accs = []
    mean_val_accs = []
    mean_train_losses = []
    mean_val_losses = []
    for epoch in tqdm(range(num_epochs), desc="Epocs"):
        model.train()
        accs = []
        val_accs = []
        losses = []
        val_losses = []
        for sample, label in tqdm(tdl, desc="Batch TRAIN"):
            opt.step()
            sample = sample.to(device)
            logits = model(sample)

            loss = F.cross_entropy(logits, label)
            losses.append(loss.item())
            acc = (logits.argmax(axis=1) == label).float().mean().cpu().numpy()
            accs.append(acc)

            opt.zero_grad()
            loss.backward()
        
        print(f"Train accuracy: {np.mean(accs)}")
        print(f"Train loss: {np.mean(losses)}")
        mean_train_accs.append(np.mean(accs))
        mean_train_losses.append(np.mean(losses))
        
        model.eval()
        
        for sample, label in tqdm(vdl, desc="Batch VAL"):
            sample = sample.to(device)
            logits = model(sample)
            loss = F.cross_entropy(logits, label)
            val_losses.append(loss.item())
            acc = (logits.argmax(axis=1) == label).float().mean().cpu().numpy()
            val_accs.append(acc)

        print(f"Val loss: {np.mean(val_losses)}")
        print(f"Val accuracy {np.mean(val_accs)}")
        mean_val_accs.append(np.mean(val_accs))
        mean_val_losses.append(np.mean(val_losses))
    return mean_train_accs, mean_val_accs, mean_train_losses, mean_val_losses


            

In [214]:
ta, va, tl, vl = train_eval_loop(model, optimizer, train_loader, val_loader)

Batch: 953it [02:24,  6.59it/s]                                                                                                                                  | 0/5 [00:00<?, ?it/s]


Train accuracy: 0.18932496011257172
Train loss: 3.2116410547138385


Epocs:  20%|████████████████████████████                                                                                                                | 1/5 [02:38<10:34, 158.56s/it]

Val loss: 3.04071447428535
Val accuracy 0.23417681455612183


Batch: 953it [02:21,  6.72it/s]


...

Val loss: 2.912301748740573
Val accuracy 0.2560547888278961


Batch: 953it [02:05,  7.57it/s]


Train accuracy: 0.30991581082344055
Train loss: 2.6478068893628754


Epocs: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [12:50<00:00, 154.14s/it]

Val loss: 2.9306278108548716
Val accuracy 0.2566072344779968


In [218]:
test_dataset = HPAC_Dataset(test_df)


In [33]:
def get_predictions(test_dataset):
    preds = []
    gt = []
    for s, l in test_dataset:
        s.to(device)
        pred = model(s.unsqueeze(0)).argmax().item()
        preds.append(pred)
        gt.append(l)
    return gt, preds


In [229]:
from sklearn.metrics import f1_score, accuracy_score

gt, preds = get_predictions(test_dataset=test_dataset)
print("NO AUG")
print(f"F1 MICRO: {f1_score(gt, preds, average='micro')}")
print(f"F1 MACRO: {f1_score(gt, preds, average='macro')}")
print(f"ACC: {accuracy_score(gt, preds)}")



NO AUG
F1 MICRO: 0.26370621174632114
F1 MACRO: 0.06273495812129097
ACC: 0.26370621174632114


In [234]:
model_f = model

## Augmetations

In [27]:
train_dataset = HPAC_Dataset(train_df, aug=True)
val_dataset = HPAC_Dataset(dev_df, aug=True)

In [28]:
import torch

model = CNNModel()
optimizer = optim.Adam(model.parameters())
device = 'cpu'  if torch.cuda.is_available() else torch.cuda.current_device()

model.to(device)
    

CNNModel(
  (model): Sequential(
    (0): Conv1d(256, 100, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(100, 16, kernel_size=(3,), stride=(1,), padding=(3,))
    (3): ReLU()
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=1664, out_features=256, bias=True)
    (6): ReLU()
    (7): Linear(in_features=256, out_features=85, bias=True)
  )
)

In [29]:
BATCH_SIZE = 64

train_loader = DataLoader(train_dataset, pin_memory=True, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_dataset, shuffle=False, pin_memory=True, batch_size=BATCH_SIZE)
    

In [30]:
ta, va, tl, vl = train_eval_loop(model, optimizer, train_loader, val_loader)

Epocs:   0%|                                                                                                                                                                                                                     | 0/5 [00:00<?, ?it/s]/home/paperspace/.local/lib/python3.8/site-packages/gensim/models/fasttext.py:1128: RuntimeWarning: invalid value encountered in divide
  return word_vec / np.linalg.norm(word_vec)
Batch TRAIN: 953it [03:06,  5.10it/s]


Train accuracy: 0.12395699322223663
Train loss: 3.4683674675219955


Batch VAL: 119it [00:16,  7.00it/s]
Epocs:  20%|████████████████████████████████████████▊                                                                                                                                                                   | 1/5 [03:23<13:35, 203.93s/it]

Val loss: 3.4690754493745435
Val accuracy 0.12195774912834167


Batch TRAIN: 953it [02:57,  5.37it/s]


...

Val loss: 3.4654005194912436
Val accuracy 0.12195774912834167


Batch TRAIN: 953it [02:56,  5.41it/s]


Train accuracy: 0.12325198203325272
Train loss: 3.4459221665781667


Batch VAL: 119it [00:14,  8.19it/s]
Epocs: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [16:02<00:00, 192.50s/it]

Val loss: 3.4649450538539086
Val accuracy 0.12195774912834167


In [31]:
test_dataset = HPAC_Dataset(test_df)

In [34]:
from sklearn.metrics import f1_score, accuracy_score

gt, preds = get_predictions(test_dataset=test_dataset)
print("AUG")
print(f"F1 MICRO: {f1_score(gt, preds, average='micro')}")
print(f"F1 MACRO: {f1_score(gt, preds, average='macro')}")
print(f"ACC: {accuracy_score(gt, preds)}")



AUG
F1 MICRO: 0.11915614012241177
F1 MACRO: 0.0025349911898666843
ACC: 0.11915614012241177


Не фортануло, что сказать